# Intro
After selecting the topology in [Amplifier Classes](./amplifier.ipynb), it's now time to build it using real components.  This notebook will go through the process of selecting components and building the amplifier.

# Transistor selection
## Limitations
* Transistor input capacitance Cin = Ciss = Cgs + Cgd
* Transistor output capacitance Cout = Coss = Cds + Cgd

## Comparison
### Characteristics
IRLML2060TRPBF
* 60V, 1.2A
* SOT23-3 package can be hand soldered.
* Ciss = 64pF
* Coss = 13pF
* Gate Charge (Qg) (Max) @ Vgs : 0.67 nC @ 4.5 V
* tr = 3.8ns, tf = 2.8ns
* Needs protection circuitry to handle over voltage when load is short-circuited.

IRLML0100TRPbF
* 100V, 1.6A
* SOT23-3
* tr = 2.1ns, tf = 3.6ns 
* Ciss = 290pF
* Coss = 27pF
* Qg = 2.5nC

#### Performance
##### IRLML2060
* Gate current = 62mA (5V)
* RF-choke current = 346mA (12V)
* 50ohm load current = 272mA
Input power = 4.46W
Output power = 3.7W
Efficiency = 83%

##### IRLML0100
* Gate current = 137mA
* RF-choke current = 315mA
* 50ohm load current = 255mA
Input power = 4.47W
Output power = 3.25W
Efficiency = 73%

### Power rating measurement using LTSpice
1. Hold Ctrl+Alt when clicking the transformer.  A temperature symbol will show up.
2. Click the temperature symbol.  This will add the instantaneous power trace to the simulation window.
3. Hover over the trace name.  When the cursor changes to a hand, Ctrl+click.
4. **The average power will be displayed : 339mW.**  Which is below the 1.25W rating of the IRLML2060.

For more info, check [LTspice: Computing the Average or RMS Value of a Trace](https://www.analog.com/en/technical-articles/ltspice-computing-the-average-or-rms-value-of-a-trace.html).

## Conclusion
The IRLML0100 can handle higher drain voltages, but simulations show that this part will break too when output is short circuited on an unprotected amplifier.  The IRLML0100 is less efficient, so there's little reason to choose this part over the IRLML2060.

# Capacitor selection : C1 & Cf
## Efficiency simulation
Vary Cf over a range of capacitance values to see how efficiency changes.  The simulation is done with a 50ohm load and 12V supply.
Previous simulations (not shown) indicated that **39pF was a good value for C1**. 
The simulations indicate that **Cf should be 82pF (more output power) to 100pF (better efficiency)**.

![Efficiency schematic](./doc/tmatchnotch_implementation_schem_Cser_efficiency.png)
![Efficiency simulation results](./doc/tmatchnotch_implementation_schem_Cser_efficiency_output.png)

The simulation shows that the efficiency is highest when Cf is 100pF.

For more info, check out [LTspice: Using .MEAS and .STEP Commands to Calculate Efficiency](https://www.analog.com/en/technical-articles/ltspice-using-meas-and-step-commands-to-calculate-efficiency.html).

## Requirements
* Voltage rating: ≥ 100V
* For temperature stability, use ceramic capacitors NP0/C0G.
* package size: 0603
* Tolerance: ± 5%

## Part selection
### C1
C1 experiences 40Vpk signals, so it should be rated for 100V.  

* Walsin 0603N390J101CT : 100V, 39pF, ±5%, NP0, 0603
* Murata GCM1885C2A390JA16D : 100V, 39pF, ±5%, NP0, 0603

### Cf & C2
Cf experiences 49Vpk signals, so it should be rated for at least 100V.  V(C2) is almost 60Vpk.  Let's take a 200V part to be safe.  Cf and C2 will be the same component type.

* Yageo CC0603JRNPOABN101 : 200V, 100pF, ±5%, NP0, 0603
* Wuert 885342006003 : 250V, 100pF, ±5%, NP0, 0603

### C3
Experiences 55Vpk signals, so it should be rated for 200V.

* Johanson 251R14S120JV4T : 250V, 12pF, ±5%, NP0, 0603
* Kyocera 06032U120FAT2A : 200V, 12pF, ±5%, NP0, 0603

# RF choke selection
Inductors will be placed in series to get to the required 10µH.  2x 4.7µH in series or 3x 3.3µH in series.  A single 10µH inductor with a self-resonant frequency greater than 100MHz is not available.

## Requirements
* Current rating > 1A
* SRF >100MHz
* Inductance L > 3µH
* package 1210

## Part choice
* Taiyo Yuden BRL3225T4R7M : 4.7µH
* Taiyo Yuden BRL3225T3R3M : 3.3µH
* Taiyo Yuden CBC3225T3R3MR : 3.3µH

# Filter inductor selection
![Filter inductor schematic](./doc/tmatchnotch_implementation_schem_L3_efficiency.png)
![Filter inductor simulation results](./doc/tmatchnotch_implementation_schem_L3_efficiency_output.png)

The filter inductor value is 650nH, which is not a standard value.  It's implemented by a 1.2µH and 1.5µH inductor in parallel.
Simulation results show that the inductor tolerance shouldn't be more than 10%.

## Filter performance
![Filter perfomance](./doc/tmatchnotch_waveforms_filterperformance.png)
### Frequency peaks
* 26.88MHz : 22.9dB (fundamental frequency)
* 53.76MHz : -27.0dB (49.9dB below fundamental)
* 80.65MHz : -21.5dB (44.4dB below fundamental)
* 107.53MHz : -30.1dB (53dB below fundamental) → in broadcast-FM band
* 134.41MHz : -32.8dB (55.7dB below fundamental)

## Part choice
### Ltot
800mApk current peaks.

* Wuerth 744766901 : 1.0µH, ±10%, 1.9A, 1812
* Epcos B82432T1102K000 : 1.0µH, ±10%, 1.3A, 1812

### L3a
* Wuerth 7447669012 : 1.2µH, ±10%, 1.8A, 1812
* Bourns CC453232-1R2KL : 1.2µH, ±10%, 1A, 1812

### L3b
* Bourns CC453232A-1R5KL : 1.5µH, ±10%, 1.9A, 1812

# Protection

Needed in case of open load or shorted load.
## MOSFET-overvoltage protection
* TVS has too much capacitance.  Varistor and TVS have a too big difference between the breakdown voltage (start of clamping action) and the maximum clamping voltage.  
* Varistors : The maximum clamping voltage is about 60% to 100% higher than the breakdown voltage.
* TVS-diode : maximum clamping voltage is about 50% higher than the breakdown voltage.  Breakdown voltage is 20% above rated voltage.
